In this live coding exercise, we will implement the formulation we derived for the given problem statement:\
Design a process with minimum cost to produce 10 kg/h of B from feedstock A (maximum availability = 15 kg/h). <br>
There are three reactors available each with different yields and costs. <br>
The details are:
| Reactor   | Yield | Fixed cost ($/h) | Variable cost ($/kg) |
|-----------|-------|------------------|-----------------------|
| Reactor 1 | 0.80  | 80               | 35                    |
| Reactor 2 | 0.667 | 54               | 30                    |
| Reactor 3 | 0.555 | 27               | 25                    |


We derived the following formulation in the lecture:
\begin{align*}
\min_{\mathbf{x,y}} & \ 80y_1 + 35x_1 + 54y_2 + 30x_2 + 27y_3 + 25x_3 \\
\text{s.t.} & \ 0.8x_1 + 0.667x_2 + 0.555x_3 = 10 \\
& \ \sum_{i=1}^3 x_i \leq 15 \\
& \ x_i - 15y_i \leq 0, \, \forall \, i \, \in \, \{1,2,3\} \\
& \ \mathbf{x} \geq 0, \mathbf{y} \in \{0,1\}
\end{align*}

Now we will implement this formulation to solve this mixed-integer linear programming problem using SciPy.

Let us import the relevant libraries first.

In [1]:
import numpy as np
from scipy.optimize import milp, Bounds, LinearConstraint

Now we will define the objective function based on the variables in our problem formulation.

In [2]:
# Coefficients for the objective function
c = np.array([35, 30, 25, 80, 54, 27])

Now we will define the matrix of coefficients of all the inequality constraints in our formulation

In [3]:
# Define the inequality constraints (Ax <= b)
A_ub = np.array([
    [1, 1, 1, 0, 0, 0],       # x1 + x2 + x3 <= 15
    [1, 0, 0, -15, 0, 0],     # x1 - 15y1 <= 0
    [0, 1, 0, 0, -15, 0],     # x2 - 15y2 <= 0
    [0, 0, 1, 0, 0, -15]      # x3 - 15y3 <= 0
])
b_ub = np.array([15, 0, 0, 0])

Now we will define the matrix of coefficients of all the equality constraints in our formulation

In [4]:
# Define the equality constraints (Ax = b)
A_eq = np.array([
    [0.8, 0.667, 0.555, 0, 0, 0]   # 0.8x1 + 0.667x2 + 0.555x3 = 10
])
b_eq = np.array([10])

Now we will specify the bounds on alll the variables in the formulation

In [5]:
# Define the bounds for the variables
x_bounds = [(0, None)] * 3    # x1, x2, x3 >= 0
y_bounds = [(0, 1)] * 3       # y1, y2, y3 are binary (0 or 1)
bounds = Bounds([0, 0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, 1, 1, 1])

Since $y$ variables are binary in nature, we need to ensure this in our formulation. We do this by defining a numpy array which tells us whether the variable is discrete or not?

In [6]:
# Define the integrality constraints
integrality = np.array([0, 0, 0, 1, 1, 1])

Now we will solve the formulation using the mixed-integer linear programming functionality of SciPy

In [7]:
# Solve the MILP problem using scipy.optimize.milp
result = milp(
    c=c,
    constraints=[
        LinearConstraint(A_ub, -np.inf, b_ub),
        LinearConstraint(A_eq, b_eq, b_eq)
    ],
    bounds=bounds,
    integrality=integrality
)

Display the results.

In [8]:
# Check the result and print the output
if result.success:
    print("Status:", result.message)
    print("Objective value:", result.fun)
    print("Variable values:", result.x)
else:
    print("Optimization failed.")
    print("Status:", result.message)

Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Objective value: 503.7751124437781
Variable values: [ 0.         14.99250375  0.         -0.          1.         -0.        ]


Inspecting the solution in more detail.

In [9]:
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 503.7751124437781
              x: [ 0.000e+00  1.499e+01  0.000e+00 -0.000e+00  1.000e+00
                  -0.000e+00]
 mip_node_count: 1
 mip_dual_bound: 503.7751124437781
        mip_gap: 0.0